# CheXpert Classification

### Dependencies

In [1]:
import tensorflow as tf
import fiftyone as fo
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

### Variables

In [2]:
epochs = 20
batch_size = 32
learning_rate = 0.0001
max_data = 50000

dataset_dir = "D:/Arthur/TIPE/data"
img_size = (224, 224, 1)
big_batch_size = 320

# Classes (only 5 of them)
classes = ["Cardiomegaly", 
           "Edema",
           "Consolidation", 
           "Atelectasis", 
           "Pleural Effusion"]

# Preparation

## Data Fetching

### 'deprecated' - Load the entire dataset

### 'done' - Preprocessing data

### Create DataGenerator

#### 'deprecated' - Directly loading images

#### Loading *.npz files

In [3]:
# Variante pour les fichiers npz

class DataGenerator(tf.keras.utils.Sequence):
    """
    Load data from pre-processed *.npz files (normalized, etc.)
    """
    
    def __init__(self, batch_size, dataset_directory):
        self.batch_size = batch_size
        self.directory = dataset_directory
        self.list_IDs = os.listdir(self.directory)
   
    def __len__(self):
        return min(len(self.list_IDs) * big_batch_size, max_data) // self.batch_size

    def __getitem__(self, index):
        
        big_batch = (index * self.batch_size) // big_batch_size
        
        np_file = np.load(self.directory+"/batch_{}.npz".format(big_batch))
        
        return np_file["images"][(index*self.batch_size)%big_batch_size : ((index*self.batch_size)%big_batch_size)+self.batch_size], \
               np_file["targets"][(index*self.batch_size)%big_batch_size : ((index*self.batch_size)%big_batch_size)+self.batch_size]
        
TrainingGenerator = DataGenerator(batch_size=batch_size, dataset_directory=dataset_dir+"/NPZ/train")
ValidationGenerator = DataGenerator(batch_size=batch_size, dataset_directory=dataset_dir+"/NPZ/valid")

## Model

#### Classification Head

In [4]:
class Classification_Head(tf.keras.Model):
    
    def __init__(self):
        super(Classification_Head, self).__init__()
        
        self.list_layers = [
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(4096, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(4096, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(len(classes), activation='sigmoid')
        ]
        
    def call(self, x):
        
        for layer in self.list_layers :
            x = layer(x)
            
        return x   

#### AlexNet

#### VGG16

### Compile functions

In [5]:
#learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=1000, decay_rate=0.96, staircase=True)
def compile(model, lr) :
    
    global loss_object
    loss_object = tf.keras.losses.BinaryCrossentropy()
    global optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # For graph mode
    global train_loss
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    global valid_loss
    valid_loss = tf.keras.metrics.Mean(name='valid_loss')
    global train_accuracy
    train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
    global valid_accuracy
    valid_accuracy = tf.keras.metrics.CategoricalAccuracy(name='valid_accuracy')

    model.compile(
            loss=loss_object,
            optimizer=optimizer,
            metrics=["accuracy"])

## Training

### Training functions

#### Step functions

In [6]:
#@tf.function
def train_step(images, targets):

    train_vars = model.trainable_variables
    accum_gradient = [tf.zeros_like(var) for var in train_vars]
  
    for image, target in zip(images, targets) :
        
        with tf.GradientTape() as tape:
            prediction = model(image)
            target.shape = model(image).shape
            loss = loss_object(target, prediction)
        
        train_loss(loss)
        train_accuracy(target, prediction) 

        gradients = tape.gradient(loss, train_vars)
        accum_gradient = [(acum_grad+grad) for acum_grad, grad in zip(accum_gradient, gradients)]
    
    accum_gradient = [grad/len(images) for grad in accum_gradient]
    
    optimizer.apply_gradients(zip(accum_gradient, model.trainable_variables))

#@tf.function
def valid_step(images, targets):
    
    predictions = model(images)
    loss = loss_object(targets, predictions)
    
    valid_loss(loss)
    valid_accuracy(targets, predictions)

#### Boucle

In [14]:
def train(model):

    b = 0 # Permet de tracker où on en est dans les batchs
    history = {'accuracy':[], 'loss':[], 'val_accuracy':[], 'val_loss':[]}
    for epoch in range(epochs):

        # TRAINING
        for images, targets in TrainingGenerator:

            train_step(images, targets)

            template = '\r Batch {}/{}, Loss: {}, Accuracy: {}'
            print(template.format(b, len(TrainingGenerator), train_loss.result(), train_accuracy.result()*100), end="")

            b += batch_size

        # VALIDATION
        for images, targets in ValidationGenerator:
            valid_step(images, targets)

        template = '\nEpoch {}, Valid Loss: {}, Valid Accuracy: {}'
        print(template.format(
            epoch+1,
            valid_loss.result(), 
            valid_accuracy.result()*100)
        )
        
        # On stocke les accumulateurs dans l'history
        history['accuracy'].append(train_accuracy.result())
        history['loss'].append(train_loss.result())
        history['val_accuracy'].append(valid_accuracy.result())
        history['val_loss'].append(valid_loss.result())
        
        # On reset les accumulateurs
        valid_loss.reset_states()
        valid_accuracy.reset_states()
        train_accuracy.reset_states()
        train_loss.reset_states()
        b = 0
        print("")
    
    return history

### Post-training functions

#### save_training

In [8]:
def save_training(history) :

    os.chdir(dataset_dir+"/training_sessions")

    # Create a folder for the training session
    i=0
    error = True
    while error:
        try:
            fd = "{}_{}k_{}-epochs_{}".format(model.name, str(max_data)[:-3], str(epochs), str(i))
            os.mkdir("{}".format(fd))
        except FileExistsError:
            i+=1
        else:
            error=False

    os.chdir(dataset_dir+"/training_sessions/{}".format(fd))

    # Report file
    with open("training_report.txt", "w") as f:
        f.write("TRAINING REPORT :\n\n")
        f.write("epochs = " + str(epochs))
        f.write("\nbatch_size = " + str(batch_size))
        f.write("\nlearning_rate = " + str(learning_rate))
        f.write("\nmax_data = " + str(max_data))
        f.write("\n")
        f.write("\nSummary :\n")
        model.summary(print_fn=lambda x : f.write("\n" + x))

    # Retrieve curves
    loss_curve = history["loss"]
    acc_curve = history["accuracy"]
    loss_val_curve = history["val_loss"]
    acc_val_curve = history["val_accuracy"]

    # Loss fig
    plt.plot(loss_curve, "r", label="Train")
    plt.plot(loss_val_curve, "g", label="Validation")
    plt.legend(loc='upper left')
    plt.title("Loss")
    plt.savefig("loss.png")
    plt.clf()

    # Accuracy fig
    plt.plot(acc_curve, "r", label="Train")
    plt.plot(acc_val_curve, "g", label="Validation")
    plt.legend(loc='upper left')
    plt.title("Accuracy")
    plt.savefig("accuracy.png")
    plt.clf()

    # Save the model
    model.save("model.h5")

    print("# Model Saved ! #")

#### visualize_training

In [9]:
def visualize_training(history) :
    
    # Retrieve curves
    loss_curve = history["loss"]
    acc_curve = history["accuracy"]

    loss_val_curve = history["val_loss"]
    acc_val_curve = history["val_accuracy"]

    ### Plot the fig ###

    # Loss
    plt.plot(loss_curve, "r", label="Train")
    plt.plot(loss_val_curve, "g", label="Validation")
    plt.legend(loc='upper left')
    plt.title("Loss")

    plt.show()

    # Accuracy
    plt.plot(acc_curve, "r", label="Train")
    plt.plot(acc_val_curve, "g", label="Validation")
    plt.legend(loc='upper left')
    plt.title("Accuracy")

    plt.show()

# LAB

In [10]:
pretrained_extractor = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
classification_head = Classification_Head()

In [11]:
class model(tf.keras.Model):
    
    def __init__(self):
        super(model, self).__init__()
        
        self.base = pretrained_extractor
        self.classification = classification_head
        
        self.base.trainable = False
        
    def call(self, x):
        
        x = tf.image.grayscale_to_rgb(x)
        x = tf.reshape(x, shape=(-1,224,224,3))
        x = self.base(x)
        x = self.classification(x)
            
        return x

In [12]:
model = model()
compile(model, learning_rate)

In [13]:
history = train(model)

save_training(history1)
visualize_training(history1)

 Batch 0/1562, Loss: 0.8070566058158875, Accuracy: 15.625

ResourceExhaustedError: failed to allocate memory [Op:AddV2]

### Loading previsous models

In [ ]:
#loaded_model = tf.keras.models.load_model("*.h5")
len(TrainingGenerator)